In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, r2_score, confusion_matrix, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

# LECTURA Y VISUALIZACIÓN DE LOS DATOS

In [ ]:
# Cargar los datos
hotel1 = pd.read_csv("H1.csv")
hotel2 = pd.read_csv("H2.csv")

In [ ]:
# Reemplazar valores vacíos o nulos a "0"
hotel1.fillna(0, inplace=True)
hotel2.fillna(0, inplace=True)
# Conversión de variables categóricas en numéricas
hotel1 = pd.get_dummies(hotel1)
hotel2 = pd.get_dummies(hotel2)

In [ ]:
hotel1.head()

In [ ]:
hotel2.head()

# EVALUACIÓN CON EL MODELO DE LOGÍSTICA VAINILLA 

In [ ]:
# Evaluación de la regresión logística vainilla
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train1, X_test1, y_train1, y_test1 = train_test_split(hotel1.drop('IsCanceled', axis=1), hotel1['IsCanceled'], test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(hotel2.drop('IsCanceled', axis=1), hotel2['IsCanceled'], test_size=0.2, random_state=42)

In [ ]:
# Crear un modelo de regresión logística vainilla y ajustarlo a los datos de entrenamiento
logreg_H1 = LogisticRegression()
logreg_H1 = logreg_H1.fit(X_train1, y_train1)

In [ ]:
logreg_H2 = LogisticRegression()
logreg_H2 = logreg_H2.fit(X_train2, y_train2)

In [ ]:
# Evaluar el modelo en los datos de prueba
logreg_y_pred1 = logreg_H1.predict(X_test1)
logreg_y_pred2 = logreg_H2.predict(X_test2)

In [ ]:
# Calcular la precisión del modelo
print("Precisión del modelo Vainilla en H1: ", accuracy_score(y_test1, logreg_y_pred1))
print("Precisión del modelo Vainilla en H2: ", accuracy_score(y_test2, logreg_y_pred2))

# EVALUACIÓN CON EL MODELO LINEAL

In [ ]:
# Evaluar el modelo lineal
# Crear un modelo de regresión lineal y ajustarlo a los datos de entrenamiento
linreg_H1 = LinearRegression()
linreg_H1 = linreg_H1.fit(X_train1, y_train1)

In [ ]:
linreg_H2 = LinearRegression()
linreg_H2 = linreg_H2.fit(X_train2, y_train2)

In [ ]:
# Evaluar el modelo en los datos de prueba
linreg_y_pred1 = linreg_H1.predict(X_test1)
linreg_y_pred2 = linreg_H2.predict(X_test2)

In [ ]:
# Calcular el coeficiente de determinación del modelo
print("Coeficiente de determinación del modelo regresión lineal en H1: ", r2_score(y_test1, linreg_y_pred1))
print("Coeficiente de determinación del modelo regresión lineal en H2: ", r2_score(y_test2, linreg_y_pred2))

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo regresión lineal en H1:", mean_squared_error(y_test1, linreg_y_pred1))
print("MSE (Error cuadrático medio) del modelo regresión lineal en H2:", mean_squared_error(y_test2, linreg_y_pred2))

# IMPLEMENTACIÓN DEL MÉTODO SAFE

### MODELO DE BOSQUE ALEATORIO SUPERVISOR FLEXIBLE

In [ ]:
pip install safe-transformer

In [ ]:
from SafeTransformer import SafeTransformer
# Seleccionar las columnas numéricas para transformar
numeric_cols = ['IsCanceled','LeadTime','IsRepeatedGuest','PreviousBookingsNotCanceled','BookingChanges','DaysInWaitingList','RequiredCarParkingSpaces','TotalOfSpecialRequests']

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train1_SAFE, X_test1_SAFE, y_train1_SAFE, y_test1_SAFE = train_test_split(hotel1[(numeric_cols)], hotel1['IsCanceled'], test_size=0.2, random_state=42)
X_train2_SAFE, X_test2_SAFE, y_train2_SAFE, y_test2_SAFE = train_test_split(hotel2[(numeric_cols)], hotel2['IsCanceled'], test_size=0.2, random_state=42)

In [ ]:
# Entrenar un modelo complejo (bosque aleatorio supervisor flexible)
surrogate_model_H1 = RandomForestClassifier(n_estimators=100,
                                          max_depth=4,
                                          max_features='sqrt',
                                          random_state=42)
surrogate_model_H1 = surrogate_model_H1.fit(X_train1_SAFE, y_train1_SAFE)

In [ ]:
# Entrenar un modelo complejo (bosque aleatorio supervisor flexible)
surrogate_model_H2 = RandomForestClassifier(n_estimators=100,
                                          max_depth=4,
                                          max_features='sqrt',
                                          random_state=42)
surrogate_model_H2 = surrogate_model_H2.fit(X_train2_SAFE, y_train2_SAFE)

### TRANSFORMACIÓN DE VARIABLES

In [ ]:
# Utilizar SAFE para encontrar transformaciones de variables
safe_transformer_H1 = SafeTransformer(surrogate_model_H1)
X_transformed_H1 = safe_transformer_H1.fit_transform(X_train1_SAFE, y_train1_SAFE)

In [ ]:
safe_transformer_H2 = SafeTransformer(surrogate_model_H2)
X_transformed_H2 = safe_transformer_H2.fit_transform(X_train2_SAFE, y_train2_SAFE)

In [ ]:
# Realizar selección de características en el nuevo conjunto de características
linear_model_H1 = LogisticRegression()
pipe_H1 = Pipeline(steps=[('linear', linear_model_H1)])
pipe_H1 = pipe_H1.fit(X_transformed_H1, y_train1_SAFE)

In [ ]:
linear_model_H2 = LogisticRegression()
pipe_H2 = Pipeline(steps=[('linear', linear_model_H2)])
pipe_H2 = pipe_H2.fit(X_transformed_H2, y_train2_SAFE)

In [ ]:
# Evaluar el modelo de regresión logística basado en las variables transformadas
X_test_transformed_H1 = safe_transformer_H1.transform(X_test1_SAFE)
predictions_H1 = pipe_H1.predict(X_test_transformed_H1)
accuracy_H1 = accuracy_score(y_test1_SAFE, predictions_H1)

In [ ]:
X_test_transformed_H2 = safe_transformer_H2.transform(X_test2_SAFE)
predictions_H2 = pipe_H2.predict(X_test_transformed_H2)
accuracy_H2 = accuracy_score(y_test2_SAFE, predictions_H2)

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo de Bosque Aleatorio en H1:", mean_squared_error(y_test1_SAFE, predictions_H1))
print("MSE (Error cuadrático medio) del modelo de Bosque Aleatorio en H2:", mean_squared_error(y_test2_SAFE, predictions_H2))

In [ ]:
# Calculando el R2
print("Coeficiente de determinación del modelo en Bosque Aleatorio H1:", r2_score(y_test1_SAFE, predictions_H1))
print("Coeficiente de determinación del modelo en Bosque Aleatorio H2:", r2_score(y_test2_SAFE, predictions_H2))

# EVALUACIÓN CON EL MODELO DE REGRESIÓN LOGÍSTICA 

In [ ]:
# Entrenar un modelo de regresión logística con las variables transformadas
logistic_model = LogisticRegression(penalty='l2', C=0.1, solver='liblinear')
LM_H1 = logistic_model.fit(X_transformed_H1, y_train1_SAFE)
LM_H2 = logistic_model.fit(X_transformed_H2, y_train2_SAFE)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
y_pred_H1 = LM_H1.predict(X_test_transformed_H1)
y_pred_H2 = LM_H2.predict(X_test_transformed_H2)

In [ ]:
# Calcular el coeficiente de determinación del modelo
print("Coeficiente de determinación del modelo Logístico en H1: ", r2_score(y_test1_SAFE, predictions_H1))
print("Coeficiente de determinación del modelo Logístico en H2: ", r2_score(y_test2_SAFE, predictions_H2))

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo Logístico en H1:", mean_squared_error(y_test1_SAFE, predictions_H1))
print("MSE (Error cuadrático medio) del modelo Logístico en H2:", mean_squared_error(y_test2_SAFE, predictions_H2))

In [ ]:
# Imprimir la matriz de confusión
print("Matriz de confusión del modelo Logístico en H1:\n", confusion_matrix(y_test1_SAFE, predictions_H1))
print("Matriz de confusión del modelo Logístico en H2:\n", confusion_matrix(y_test2_SAFE, predictions_H2))

In [ ]:
# Calcular la precisión del modelo
print("Precisión del modelo Logístico en H1: ", accuracy_score(y_test1_SAFE, predictions_H1))
print("Precisión del modelo Logístico en H2: ", accuracy_score(y_test2_SAFE, predictions_H2))